# Part 2: Testing ML Models

In this tutorial notebook, we'll examine some examples of why we may want to test models, and how this can be done.

In [1]:
from inspect import getsource

import classic_tests
import physicality_tests

In [6]:
print(getsource(classic_tests.test_prediction_bounds))

@pytest.mark.parametrize(
    "material",
    [
        "Li7La3(SnO6)2",
        "Li3(WO3)8",
        "Li4Mn2P4H3O16",
        "Li2Ni(PO3)5",
        "MgV2O4",
    ]
)
def test_prediction_bounds(material:str, model:SynthesizabilityModel) -> None:
    predicted_value = model.predict_single(material)
    assert 0 <= predicted_value <= 1



In [7]:
print(getsource(classic_tests.test_invalid_material))

@pytest.mark.parametrize(
    "material",
    [
        1234,
        dict({"a":"b"}),
        ["Fe2O3"],
        ["Fe2", "O3"],
    ]
)
def test_invalid_material(material:str, model:SynthesizabilityModel) -> None:
    with pytest.raises(TypeError):
        model.predict_single(material)



In [8]:
!pytest classic_tests.py

============================= test session starts ==============================
platform darwin -- Python 3.7.6, pytest-7.1.0, pluggy-0.13.1
rootdir: /Users/eddie/Documents/GitHub/mrs-s22-ds04-tutorial/02_testing_models
plugins: hypothesis-5.5.4, arraydiff-0.3, remotedata-0.3.2, openfiles-0.4.0, doctestplus-0.5.0, astropy-header-0.1.2
collected 9 items                                                              

classic_tests.py .........                                               [100%]

============================== 9 passed in 0.05s ===============================


In [9]:
!pytest physicality_tests.py

============================= test session starts ==============================
platform darwin -- Python 3.7.6, pytest-7.1.0, pluggy-0.13.1
rootdir: /Users/eddie/Documents/GitHub/mrs-s22-ds04-tutorial/02_testing_models
plugins: hypothesis-5.5.4, arraydiff-0.3, remotedata-0.3.2, openfiles-0.4.0, doctestplus-0.5.0, astropy-header-0.1.2
collected 4 items                                                              

physicality_tests.py xxxx                                                [100%]

============================== 4 xfailed in 0.04s ==============================
